In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
#sales_data = pd.read_csv('sample_sales_data.csv')
hdat = pd.read_csv('HDAT concat3.csv')
sales_data = pd.read_csv('Iowa_Liquor_Sales.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
hdat['zipcode'] = hdat['geoTypeId'].apply(lambda x: x.split(":")[0])

In [4]:
cols = hdat.columns.values
day_cols = []

for c in hdat.columns:
    if "Day" in c:
        if 'Daytime' not in c:
            day_cols.append(c)


In [5]:
imp_cols = []
imp_cols = ['date', 'geoType', 'geoTypeId', 'latitude', 'longitude', 'zipcode']

for d in day_cols:
    imp_cols.append(d)
#imp_cols

In [6]:
len(sales_data['Store Number'].drop_duplicates())

1884

In [15]:
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

In [18]:
result_columns = ['date', 'zipcode',
       'FeelsLikeLocalDayAvg', 'FeelsLikeLocalDayMax', 'FeelsLikeLocalDayMin',
       'PrecipAmountLocalDayAvg', 'PrecipAmountLocalDayMax',
       'PrecipAmountLocalDayMin', 'SnowAmountLocalDayAvg',
       'SnowAmountLocalDayMax', 'SnowAmountLocalDayMin',
       'TemperatureLocalDayAvg', 'TemperatureLocalDayMax',
       'TemperatureLocalDayMin', 'WindSpeedLocalDayAvg',
       'WindSpeedLocalDayMax', 'WindSpeedLocalDayMin',
       'GustLocalDayAvg', 'GustLocalDayMax', 'GustLocalDayMin',
       'RelativeHumidityLocalDayAvg', 'RelativeHumidityLocalDayMax', 'RelativeHumidityLocalDayMin',
        'VisibilityLocalDayAvg', 'VisibilityLocalDayMax', 'VisibilityLocalDayMin']

sales_data = sales_data[sales_data['Date'] >= pd.to_datetime('10/1/2015')]
sales_data = sales_data[sales_data['Date']<= pd.to_datetime('10/1/2017')]
sales_data = sales_data.sort_values('Date')

stores = sales_data['Store Number'].drop_duplicates()
stores

8149861     4912
3699216     5043
6794090     2616
3697772     4429
4804513     2561
3696735     2663
4805157     4686
7609031     2587
3694106     3670
4806442     2567
3693422     5165
3693093     2571
4809454     5034
4807433     3732
455263      4345
4807455     4412
4807863     3920
751948      3888
7609838     4807
7609965     4092
6696152     4406
3509770     4269
3509581     4988
5080129     2106
4809599     3054
454618      4668
3683750     4787
8077188     5174
7433026     2806
7039627     3713
            ... 
11825893    5435
11826453    5446
11828529    5443
11831474    5445
11835458    5441
11899173    5444
11997442    5449
12049015    5453
12116913    5433
12128948    2674
12132592    5448
12136830    5458
12167143    5447
12169987    5456
12170067    5455
12201721    5465
12210943    5462
12218193    5466
12220962    5454
12230727    5464
12235714    5450
12280350    5461
12284824    5457
12305011    5379
12312414    5470
12314005    5451
12346249    5463
12349672    54

In [19]:
merged_list = []

for store in stores:
    
    view = sales_data[sales_data['Store Number'] == store]
    zipcode = str(view['Zip Code'].drop_duplicates().values[0])#.astype(str)
    weather_view = hdat[hdat['zipcode'] == zipcode]
    weather_view = weather_view[imp_cols]
    dates = view['Date'].drop_duplicates().apply(lambda x:x.strftime('%Y/%m/%d'))

    result_list = []

    for i in range(0, len(dates)-1):

        start_date = dates.iloc[i]
        end_date = dates.iloc[i+1]

        start = int(start_date.replace("/", ""))
        end = int(end_date.replace("/", ""))


        w2 = weather_view[weather_view['date']>=start]
        w2 = w2[w2['date']<end]
        w2 = w2.sort_values('date')

        interval = pd.DataFrame(index=range(1), columns=result_columns)

        for c in result_columns:

            if 'Max' in c:
                interval[c] = w2[c].max()
            elif 'Min' in c:
                interval[c] = w2[c].min()

            elif 'Avg' in c:
                interval[c] = w2[c].mean()

            elif c == 'date':
                interval[c] = end_date
            elif c == 'zipcode':
                interval[c] = zipcode
            else:
                interval[c] = w2[c].drop_duplicates()

        result_list.append(interval)
        #print("INTERVAL "+str(i)+": "+str(start)+" -- "+str(end)+ " ---> DONE")
    if len(result_list) > 0:
        result = pd.concat(result_list)
        result = result.rename(columns={'date':'Date'})
        result['Date'] = pd.to_datetime(result['Date']).apply(lambda x:x.strftime('%Y-%m-%d')).astype(str)
        view['Date'] = view['Date'].astype(str)
        merged_result = view.merge(result, on='Date', how='inner')
        merged_list.append(merged_result)
        print("STORE "+str(store)+" DONE")
    else:
        print("No records for this one")
    
final_result = pd.concat(merged_list)

final_result.to_csv('weather_sales_sample_result3.csv', index=False)
print('\n')
print("ALL STORES DONE!")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


STORE 4912 DONE
STORE 5043 DONE
STORE 2616 DONE
STORE 4429 DONE
STORE 2561 DONE
STORE 2663 DONE
STORE 4686 DONE
STORE 2587 DONE
STORE 3670 DONE
STORE 2567 DONE
STORE 5165 DONE
STORE 2571 DONE
STORE 5034 DONE
STORE 3732 DONE
STORE 4345 DONE
STORE 4412 DONE
STORE 3920 DONE
STORE 3888 DONE
STORE 4807 DONE
STORE 4092 DONE
STORE 4406 DONE
STORE 4269 DONE
STORE 4988 DONE
STORE 2106 DONE
STORE 3054 DONE
STORE 4668 DONE
STORE 4787 DONE
STORE 5174 DONE
STORE 2806 DONE
STORE 3713 DONE
STORE 4091 DONE
STORE 5162 DONE
STORE 2627 DONE
STORE 5113 DONE
STORE 5116 DONE
STORE 4969 DONE
STORE 2635 DONE
STORE 2582 DONE
STORE 2579 DONE
STORE 2190 DONE
STORE 4370 DONE
STORE 5106 DONE
STORE 4196 DONE
STORE 2652 DONE
STORE 4849 DONE
STORE 3647 DONE
STORE 2633 DONE
STORE 3664 DONE
STORE 2595 DONE
STORE 3065 DONE
STORE 5063 DONE
STORE 4260 DONE
STORE 4595 DONE
STORE 5037 DONE
STORE 4603 DONE
STORE 4214 DONE
STORE 5144 DONE
STORE 2550 DONE
STORE 2515 DONE
STORE 2248 DONE
STORE 4343 DONE
STORE 3858 DONE
STORE 25

STORE 2228 DONE
STORE 3443 DONE
STORE 2602 DONE
STORE 4924 DONE
STORE 4182 DONE
STORE 4254 DONE
STORE 4318 DONE
STORE 3514 DONE
STORE 4891 DONE
STORE 2327 DONE
STORE 2604 DONE
STORE 4651 DONE
STORE 4620 DONE
STORE 4890 DONE
STORE 3632 DONE
STORE 4393 DONE
STORE 2660 DONE
STORE 4770 DONE
STORE 2641 DONE
STORE 4337 DONE
STORE 4359 DONE
STORE 2509 DONE
STORE 5121 DONE
STORE 4357 DONE
STORE 2529 DONE
STORE 4810 DONE
STORE 4239 DONE
STORE 4150 DONE
STORE 4700 DONE
STORE 4137 DONE
STORE 4764 DONE
STORE 3989 DONE
STORE 4449 DONE
STORE 4474 DONE
STORE 4070 DONE
STORE 3477 DONE
STORE 4878 DONE
STORE 4559 DONE
STORE 4233 DONE
STORE 4904 DONE
STORE 4234 DONE
STORE 4106 DONE
STORE 4592 DONE
STORE 2205 DONE
STORE 5080 DONE
STORE 3830 DONE
STORE 4195 DONE
STORE 4609 DONE
STORE 2624 DONE
STORE 4446 DONE
STORE 3744 DONE
STORE 2597 DONE
STORE 4629 DONE
STORE 5097 DONE
STORE 4485 DONE
STORE 5088 DONE
STORE 4956 DONE
STORE 4899 DONE
STORE 2655 DONE
STORE 4731 DONE
STORE 4730 DONE
STORE 3963 DONE
STORE 37

STORE 4692 DONE
STORE 4800 DONE
STORE 5055 DONE
STORE 4718 DONE
STORE 4793 DONE
STORE 5175 DONE
STORE 4025 DONE
STORE 4285 DONE
STORE 4585 DONE
STORE 4533 DONE
STORE 3779 DONE
STORE 4980 DONE
STORE 4589 DONE
STORE 4708 DONE
STORE 4760 DONE
STORE 4249 DONE
STORE 2845 DONE
STORE 3841 DONE
STORE 4248 DONE
STORE 4634 DONE
STORE 5099 DONE
STORE 4774 DONE
STORE 4873 DONE
STORE 4326 DONE
STORE 4769 DONE
STORE 4765 DONE
STORE 4608 DONE
STORE 5110 DONE
STORE 5133 DONE
STORE 3576 DONE
STORE 4887 DONE
STORE 2848 DONE
STORE 4458 DONE
STORE 4744 DONE
STORE 4172 DONE
STORE 4256 DONE
STORE 4042 DONE
STORE 4430 DONE
STORE 3872 DONE
STORE 5134 DONE
STORE 5052 DONE
STORE 4435 DONE
STORE 4687 DONE
STORE 5058 DONE
STORE 4440 DONE
STORE 4854 DONE
STORE 4525 DONE
STORE 3688 DONE
STORE 5104 DONE
STORE 4387 DONE
STORE 5153 DONE
STORE 4853 DONE
STORE 2658 DONE
STORE 4981 DONE
STORE 4999 DONE
STORE 2238 DONE
STORE 4510 DONE
STORE 5092 DONE
STORE 4515 DONE
STORE 3880 DONE
STORE 4756 DONE
STORE 4517 DONE
STORE 45

STORE 5428 DONE
STORE 5418 DONE
STORE 5406 DONE
STORE 5417 DONE
STORE 5412 DONE
STORE 5397 DONE
STORE 5407 DONE
STORE 5408 DONE
STORE 5395 DONE
STORE 9031 DONE
STORE 5434 DONE
STORE 5432 DONE
STORE 5437 DONE
STORE 9931 DONE
STORE 5436 DONE
No records for this one
STORE 5439 DONE
STORE 5440 DONE
STORE 5435 DONE
STORE 5446 DONE
STORE 5443 DONE
STORE 5445 DONE
STORE 5441 DONE
STORE 5444 DONE
STORE 5449 DONE
No records for this one
No records for this one
STORE 2674 DONE
STORE 5448 DONE
STORE 5458 DONE
STORE 5447 DONE
STORE 5456 DONE
STORE 5455 DONE
STORE 5465 DONE
STORE 5462 DONE
STORE 5466 DONE
No records for this one
STORE 5464 DONE
No records for this one
STORE 5461 DONE
STORE 5457 DONE
STORE 5379 DONE
STORE 5470 DONE
No records for this one
No records for this one
No records for this one
No records for this one
No records for this one


ALL STORES DONE!
